In [40]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
import warnings
import yaml
from sklearn.model_selection import StratifiedKFold
from functions.models import get_ensemble_model, get_linear_model, get_nonlinear_model
from functions.models import get_model_explanations, get_age_corrected_model_explanations, correct_age_predictions
from sklearn.metrics import r2_score, mean_absolute_error
warnings.filterwarnings('ignore')
%matplotlib inline

In [14]:
df_sheet_1 = pd.read_excel('./feat_imp/total.xlsx', sheet_name='Sheet1')
df_sheet_2 = pd.read_excel('./feat_imp/total.xlsx', sheet_name='Sheet2')
df_sheet_3 = pd.read_excel('./feat_imp/total.xlsx', sheet_name='Sheet3')
df_sheet_4 = pd.read_excel('./feat_imp/total.xlsx', sheet_name='Sheet4')
df_sheet_5 = pd.read_excel('./feat_imp/total.xlsx', sheet_name='Sheet5')

In [15]:
df_sheet_3

,Unnamed: 0,HCP,Unnamed: 2,Unnamed: 3,CamCAN,Unnamed: 5,Unnamed: 6,IXI,Unnamed: 8,Unnamed: 9
0,Feature,Lasso,Gaussian Process,Gradient Boosting Regressor,Lasso,Gaussian Process,Gradient Boosting Regressor,Lasso,Gaussian Process,Gradient Boosting Regressor
1,etiv,1.24636,3.449212,0.108937,7.714287,8.831332,0.72565,4.306448,4.933209,0.225736
2,lh_accumbens_area,0.039937,0.301591,0.025911,0.391047,0.079653,0.252782,0.492026,0.258993,0.481662
3,lh_amygdala,0.000064,0.060219,0.01316,1.233872,1.531566,0.314019,0.48063,1.126055,0.034788
4,lh_bankssts_area,0.000135,0.04717,0.010552,0.480861,0.689201,0.040053,0.503797,0.700575,0.049472
...,...,...,...,...,...,...,...,...,...,...
149,rh_temporalpole_area,0.001878,0.016277,0.025055,0.103885,0.189095,0.065579,1.124799,1.080493,0.237111
150,rh_temporalpole_thickness,0.077093,0.088844,0.050545,0.63518,0.641686,0.10454,0.445646,0.45904,0.0543
151,rh_thalamus,0.000069,0.404682,0.024247,0.412486,1.21737,0.224804,0.033622,0.422258,0.324009
152,rh_transversetemporal_area,0.136983,0.204541,0.053655,0.011757,0.333466,0.053689,0.887465,1.319851,0.104478


In [16]:
feature_name = df_sheet_3.iloc[:, 0]
hcp_exp = df_sheet_3.iloc[:, 1:4]
cc_exp = df_sheet_3.iloc[:, 4:7]
ixi_exp = df_sheet_3.iloc[:, 7:]

In [17]:
hcp_exp = pd.concat([feature_name, hcp_exp], axis=1)
cc_exp = pd.concat([feature_name, cc_exp], axis=1)
ixi_exp = pd.concat([feature_name, ixi_exp], axis=1)

In [18]:
hcp_header = hcp_exp.iloc[0]
hcp_exp = hcp_exp[1:]
hcp_exp.rename(columns=hcp_header, inplace=True)
hcp_exp

,Feature,Lasso,Gaussian Process,Gradient Boosting Regressor
1,etiv,1.24636,3.449212,0.108937
2,lh_accumbens_area,0.039937,0.301591,0.025911
3,lh_amygdala,0.000064,0.060219,0.01316
4,lh_bankssts_area,0.000135,0.04717,0.010552
5,lh_bankssts_thickness,0.011672,0.114803,0.025919
...,...,...,...,...
149,rh_temporalpole_area,0.001878,0.016277,0.025055
150,rh_temporalpole_thickness,0.077093,0.088844,0.050545
151,rh_thalamus,0.000069,0.404682,0.024247
152,rh_transversetemporal_area,0.136983,0.204541,0.053655


In [19]:
cc_header = cc_exp.iloc[0]
cc_exp = cc_exp[1:]
cc_exp.rename(columns=cc_header, inplace=True)

ixi_header = ixi_exp.iloc[0]
ixi_exp = ixi_exp[1:]
ixi_exp.rename(columns=ixi_header, inplace=True)

In [22]:
hcp_exp.to_csv('./feat_imp/hcp_exp.csv')
ixi_exp.to_csv('./feat_imp/ixi_exp.csv')
cc_exp.to_csv('./feat_imp/cc_exp.csv')

## HCP

In [41]:
with open("config.yaml", 'r') as ymlfile:
    cfg = yaml.safe_load(ymlfile)
print('')
print('---------------------------------------------------------')
print('Configuration:')
print(yaml.dump(cfg, default_flow_style=False, default_style=''))
print('---------------------------------------------------------')
print('')

# set paths
datapath = cfg['paths']['datapath']
metricpath = datapath + 'surfaces/'
outpath = cfg['paths']['results']
genpath = cfg['paths']['genpath']

# other params - whether to regress out global metrics and run PCA
preprocessing_params = cfg['preproc']
regress = 'Corrected' if preprocessing_params['regress'] else 'Raw'
run_pca = 'PCA' if preprocessing_params['pca'] else 'noPCA'
run_combat = 'Combat' if preprocessing_params['combat'] else 'noCombat'

# cortical parcellation
parc = cfg['data']['parcellation']

# k-fold CV params
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42) # n_split : 5


---------------------------------------------------------
Configuration:
data:
  parcellation: CAMCAN
paths:
  datapath: C:/Users/HYUK/Desktop/Brain_Age_Revision_SHAP/data/
  genpath: C:/Users/HYUK/Desktop/Brain_Age_Revision_SHAP/data/generated_data/
  results: C:/Users/HYUK/Desktop/Brain_Age_Revision_SHAP/results/
preproc:
  combat: false
  pca: false
  pca_comps: 0.9
  regress: false

---------------------------------------------------------



In [38]:
subject_data = pd.read_csv('./data/hcp_train.csv')
n_subs = 890
n_features = 153
num_of_models = 3
num_folds = 5

In [24]:
hcp_exp = pd.read_csv('./feat_imp/hcp_exp.csv', index_col=0)
hcp_exp.head()

,Feature,Lasso,Gaussian Process,Gradient Boosting Regressor
1,etiv,1.246360,3.449212,0.108937
2,lh_accumbens_area,0.039937,0.301591,0.025911
3,lh_amygdala,0.000064,0.060219,0.013160
4,lh_bankssts_area,0.000135,0.047170,0.010552
5,lh_bankssts_thickness,0.011672,0.114803,0.025919


In [26]:
hcp_lasso = hcp_exp.loc[:, ['Feature','Lasso']]
hcp_gpr = hcp_exp.loc[:, ['Feature','Gaussian Process']]
hcp_gbm = hcp_exp.loc[:, ['Feature','Gradient Boosting Regressor']]

In [31]:
hcp_lasso_sort = hcp_lasso.sort_values(by='Lasso', ascending=False)
hcp_gpr_sort = hcp_gpr.sort_values(by='Gaussian Process', ascending=False)
hcp_gbm_sort = hcp_gbm.sort_values(by='Gradient Boosting Regressor', ascending=False)

In [34]:
hcp_lasso_feat_list = hcp_lasso_sort.Feature.to_list()
hcp_gpr_feat_list = hcp_gpr_sort.Feature.to_list()
hcp_gbm_feat_list = hcp_gbm_sort.Feature.to_list()

### HCP - LASSO

In [55]:
# Feature가 하나씩 늘어갈 때마다 어떤 식으로 Metric이 변화하는 지를 저장하는 list
uncorr_mae_list = []
uncorr_r2_list = []
corr_mae_list = []
corr_r2_list = []

for feature_num in range(1, len(hcp_lasso_feat_list) + 1):
    if feature_num % 10 ==0:
        print(f"Using {feature_num} features") 
    # for문을 돌면서 Mean Absolute SHAP value가 가장 높은 순서대로 하나씩 추가해가며 
    # Model Type 변경시 이 부분 수정 
    subject_data_iter = subject_data.loc[:, hcp_lasso_feat_list[:feature_num]]
    
    # fold마다 생성되는 mae, r2 값 저장 
    unmae_fold_list = []
    unr2_fold_list = []
    mae_fold_list = []
    r2_fold_list = []
    
    for n, (train_idx, test_idx) in enumerate(skf.split(np.arange(n_subs), subject_data.Age)):
        #print('')
        #print('FOLD {:}:------------------------------------------------'.format(n+1))
        
        # Data 선언
        train_y, test_y = subject_data.Age[train_idx], subject_data.Age[test_idx]
        # train_x, test_x = subject_data_iter.drop(['Age', 'Subject'], axis=1), subject_data_iter.drop(['Age', 'Subject'], axis=1)

        train_x = subject_data_iter.loc[train_idx]
        test_x = subject_data_iter.loc[test_idx]
        
        # Model 선언
        # Model Type 변경시 이 부분 수정 
        model = get_linear_model(preprocessing_params)
        
        # Fitting
        model.fit(train_x, train_y)
            
        # PREDICT 
        train_predictions = model.predict(train_x)
        test_predictions = model.predict(test_x)
        
        # 각 fold에서 Test sample들에 대한 Prediction
        uncorr_preds = test_predictions
        corr_preds = correct_age_predictions(train_predictions, train_y, test_predictions, test_y)
            
        # MAE, R2 계산 
        unmae_fold_list.append(mean_absolute_error(test_y, uncorr_preds))
        unr2_fold_list.append(r2_score(test_y, uncorr_preds))
        mae_fold_list.append(mean_absolute_error(test_y, corr_preds))
        r2_fold_list.append(r2_score(test_y, corr_preds))
        
    # 5 fold를 전부 다 돌았으면, 평균 MAE, R2 list에 저장 
    uncorr_mae_list.append(np.mean(unmae_fold_list))
    uncorr_r2_list.append(np.mean(unr2_fold_list))
    corr_mae_list.append(np.mean(mae_fold_list))
    corr_r2_list.append(np.mean(r2_fold_list))
    
    
hcp_lasso_metrics = {'uncorr_mae' : uncorr_mae_list, 'uncorr_r2': uncorr_r2_list, 
                    'corr_mae':corr_mae_list, 'corr_r2': corr_r2_list}
hcp_lasso_metrics = pd.DataFrame(hcp_lasso_metrics)
hcp_lasso_metrics.head()

Using 10 features
Using 20 features
Using 30 features
Using 40 features
Using 50 features
Using 60 features
Using 70 features
Using 80 features
Using 90 features
Using 100 features
Using 110 features
Using 120 features
Using 130 features
Using 140 features
Using 150 features


### HCP - GPR

In [60]:
# Feature가 하나씩 늘어갈 때마다 어떤 식으로 Metric이 변화하는 지를 저장하는 list
uncorr_mae_list = []
uncorr_r2_list = []
corr_mae_list = []
corr_r2_list = []

for feature_num in range(1, len(hcp_gpr_feat_list) + 1):
    #if feature_num % 10 ==0:
    print(f"Using {feature_num} features") 
    # for문을 돌면서 Mean Absolute SHAP value가 가장 높은 순서대로 하나씩 추가해가며 
    # Model Type 변경시 이 부분 수정 
    subject_data_iter = subject_data.loc[:, hcp_gpr_feat_list[:feature_num]]
    
    # fold마다 생성되는 mae, r2 값 저장 
    unmae_fold_list = []
    unr2_fold_list = []
    mae_fold_list = []
    r2_fold_list = []
    
    for n, (train_idx, test_idx) in enumerate(skf.split(np.arange(n_subs), subject_data.Age)):
        
        # Data 선언
        train_y, test_y = subject_data.Age[train_idx], subject_data.Age[test_idx]
        train_x = subject_data_iter.loc[train_idx]
        test_x = subject_data_iter.loc[test_idx]
        
        # Model 선언
        # Model Type 변경시 이 부분 수정 
        model = get_nonlinear_model(preprocessing_params)
        
        # Fitting
        model.fit(train_x, train_y)
            
        # PREDICT 
        train_predictions = model.predict(train_x)
        test_predictions = model.predict(test_x)
        
        # 각 fold에서 Test sample들에 대한 Prediction
        uncorr_preds = test_predictions
        corr_preds = correct_age_predictions(train_predictions, train_y, test_predictions, test_y)
            
        # MAE, R2 계산 
        unmae_fold_list.append(mean_absolute_error(test_y, uncorr_preds))
        unr2_fold_list.append(r2_score(test_y, uncorr_preds))
        mae_fold_list.append(mean_absolute_error(test_y, corr_preds))
        r2_fold_list.append(r2_score(test_y, corr_preds))
        
    # 5 fold를 전부 다 돌았으면, 평균 MAE, R2 list에 저장 
    uncorr_mae_list.append(np.mean(unmae_fold_list))
    uncorr_r2_list.append(np.mean(unr2_fold_list))
    corr_mae_list.append(np.mean(mae_fold_list))
    corr_r2_list.append(np.mean(r2_fold_list))
    


Using 1 features
Using 2 features
Using 3 features
Using 4 features
Using 5 features
Using 6 features
Using 7 features
Using 8 features
Using 9 features
Using 10 features
Using 11 features
Using 12 features
Using 13 features
Using 14 features
Using 15 features
Using 16 features
Using 17 features
Using 18 features
Using 19 features
Using 20 features
Using 21 features
Using 22 features
Using 23 features
Using 24 features
Using 25 features
Using 26 features
Using 27 features
Using 28 features
Using 29 features
Using 30 features
Using 31 features
Using 32 features
Using 33 features
Using 34 features
Using 35 features
Using 36 features
Using 37 features
Using 38 features
Using 39 features
Using 40 features
Using 41 features
Using 42 features
Using 43 features
Using 44 features
Using 45 features
Using 46 features
Using 47 features
Using 48 features
Using 49 features
Using 50 features
Using 51 features
Using 52 features
Using 53 features
Using 54 features
Using 55 features
Using 56 features
U

KeyboardInterrupt: 

In [ ]:
# Model Type 변경시 변수 명 수정 (4군데)
hcp_gpr_metrics = {'uncorr_mae' : uncorr_mae_list, 'uncorr_r2': uncorr_r2_list, 
                    'corr_mae':corr_mae_list, 'corr_r2': corr_r2_list}
hcp_gpr_metrics = pd.DataFrame(hcp_gpr_metrics)
hcp_gpr_metrics.head()

### HCP - GBM

In [ ]:
# Feature가 하나씩 늘어갈 때마다 어떤 식으로 Metric이 변화하는 지를 저장하는 list
uncorr_mae_list = []
uncorr_r2_list = []
corr_mae_list = []
corr_r2_list = []

for feature_num in range(1, len(hcp_gbm_feat_list) + 1):
    if feature_num % 10 ==0:
        print(f"Using {feature_num} features") 
    # for문을 돌면서 Mean Absolute SHAP value가 가장 높은 순서대로 하나씩 추가해가며 
    # Model Type 변경시 이 부분 수정 
    subject_data_iter = subject_data.loc[:, hcp_gbm_feat_list[:feature_num]]
    
    # fold마다 생성되는 mae, r2 값 저장 
    unmae_fold_list = []
    unr2_fold_list = []
    mae_fold_list = []
    r2_fold_list = []
    
    for n, (train_idx, test_idx) in enumerate(skf.split(np.arange(n_subs), subject_data.Age)):
        #print('')
        #print('FOLD {:}:------------------------------------------------'.format(n+1))
        
        # Data 선언
        train_y, test_y = subject_data.Age[train_idx], subject_data.Age[test_idx]
        # train_x, test_x = subject_data_iter.drop(['Age', 'Subject'], axis=1), subject_data_iter.drop(['Age', 'Subject'], axis=1)

        train_x = subject_data_iter.loc[train_idx]
        test_x = subject_data_iter.loc[test_idx]
        
        # Model 선언
        # Model Type 변경시 이 부분 수정 
        model = get_ensemble_model(preprocessing_params)
        
        # Fitting
        model.fit(train_x, train_y)
            
        # PREDICT 
        train_predictions = model.predict(train_x)
        test_predictions = model.predict(test_x)
        
        # 각 fold에서 Test sample들에 대한 Prediction
        uncorr_preds = test_predictions
        corr_preds = correct_age_predictions(train_predictions, train_y, test_predictions, test_y)
            
        # MAE, R2 계산 
        unmae_fold_list.append(mean_absolute_error(test_y, uncorr_preds))
        unr2_fold_list.append(r2_score(test_y, uncorr_preds))
        mae_fold_list.append(mean_absolute_error(test_y, corr_preds))
        r2_fold_list.append(r2_score(test_y, corr_preds))
        
    # 5 fold를 전부 다 돌았으면, 평균 MAE, R2 list에 저장 
    uncorr_mae_list.append(np.mean(unmae_fold_list))
    uncorr_r2_list.append(np.mean(unr2_fold_list))
    corr_mae_list.append(np.mean(mae_fold_list))
    corr_r2_list.append(np.mean(r2_fold_list))
    
    
hcp_gbm_metrics = {'uncorr_mae' : uncorr_mae_list, 'uncorr_r2': uncorr_r2_list, 
                    'corr_mae':corr_mae_list, 'corr_r2': corr_r2_list}
hcp_gbm_metrics = pd.DataFrame(hcp_gbm_metrics)
hcp_gbm_metrics.head()